Using a mix of `partridge` and `gtfstk` with some of my own additions to create daily statistical DataFrames for trips, routes and stops. This will later become a module which we will run on our historical MoT GTFS archive and schedule for nightly runs. 

## Imports and config

In [5]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
import pandas as pd
import numpy as np
import partridge as ptg
import datetime

from gtfs_utils import *
import gtfstk


In [7]:
LOCAL_GTFS_ZIP_PATH = 'data/gtfs_feeds/2018-03-05.zip' 
LOCAL_TARIFF_PATH = 'data/sample/latest_tariff.zip' 

In [13]:
from collections import OrderedDict

In [18]:
def compute_trip_stats_partridge(feed, zones):
    f = feed.trips
    f = (
        f[['route_id', 'trip_id', 'direction_id', 'shape_id']]
        .merge(feed.routes[['route_id', 'route_short_name', 'route_type']])
        .merge(feed.stop_times)
        .merge(feed.stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'stop_code']])
        .merge(zones)
        .sort_values(['trip_id', 'stop_sequence'])
        #.assign(departure_time=lambda x: x['departure_time'].map(
        #    hp.timestr_to_seconds)
        #       )
        )
    geometry_by_stop = gtfstk.build_geometry_by_stop(feed, use_utm=True)
    
    g = f.groupby('trip_id')
    
    def my_agg(group):
        d = OrderedDict()
        d['route_id'] = group['route_id'].iat[0]
        d['route_short_name'] = group['route_short_name'].iat[0]
        d['route_type'] = group['route_type'].iat[0]
        d['direction_id'] = group['direction_id'].iat[0]
        d['shape_id'] = group['shape_id'].iat[0]
        d['num_stops'] = group.shape[0]
        d['start_time'] = group['departure_time'].iat[0]
        d['end_time'] = group['departure_time'].iat[-1]
        d['start_stop_id'] = group['stop_id'].iat[0]
        d['end_stop_id'] = group['stop_id'].iat[-1]
        d['start_stop_code'] = group['stop_code'].iat[0]
        d['end_stop_code'] = group['stop_code'].iat[-1]
        d['start_stop_name'] = group['stop_name'].iat[0]
        d['end_stop_name'] = group['stop_name'].iat[-1]
        d['start_zone'] = group['zone_name'].iat[0]
        d['end_zone'] = group['zone_name'].iat[-1]
        dist = geometry_by_stop[d['start_stop_id']].distance(
          geometry_by_stop[d['end_stop_id']])
        d['is_loop'] = int(dist < 400)
        d['duration'] = (d['end_time'] - d['start_time'])/3600
        return pd.Series(d)
    
    h = g.apply(my_agg)
    h['distance'] = g.shape_dist_traveled.max()
    
    # Reset index and compute final stats
    h = h.reset_index()
    h['speed'] = h['distance'] / h['duration'] / 1000
    h[['start_time', 'end_time']] = (
      h[['start_time', 'end_time']].applymap(
          lambda x: gtfstk.helpers.timestr_to_seconds(x, inverse=True))
        )
    return h

def compute_route_stats_base_partridge(trip_stats_subset,
  headway_start_time='07:00:00', headway_end_time='19:00:00', *,
  split_directions=False):
    """
    Compute stats for the given subset of trips stats.

    Parameters
    ----------
    trip_stats_subset : DataFrame
        Subset of the output of :func:`.trips.compute_trip_stats`
    split_directions : boolean
        If ``True``, then separate the stats by trip direction (0 or 1);
        otherwise aggregate trips visiting from both directions. 
        Default: ``False``
    headway_start_time : string
        HH:MM:SS time string indicating the start time for computing
        headway stats
        Default: ``'07:00:00'``
    headway_end_time : string
        HH:MM:SS time string indicating the end time for computing
        headway stats.
        Default: ``'19:00:00'``

    Returns
    -------
    DataFrame
        Columns are

        - ``'route_id'``
        - ``'route_short_name'``
        - ``'agency_id'``
        - ``'agency_name'``
        - ``'route_long_name'``
        - ``'route_type'``
        - ``'direction_id'``: 1/0
        - ``'num_trips'``: number of trips on the route in the subset
        - ``'num_trip_starts'``: number of trips on the route with
          nonnull start times
        - ``'num_trip_ends'``: number of trips on the route with nonnull
          end times that end before 23:59:59
        - ``'is_loop'``: 1 if at least one of the trips on the route has
          its ``is_loop`` field equal to 1; 0 otherwise
        - ``'is_bidirectional'``: 1 if the route has trips in both
          directions; 0 otherwise
        - ``'start_time'``: start time of the earliest trip on the route
        - ``'end_time'``: end time of latest trip on the route
        - ``'max_headway'``: maximum of the durations (in minutes)
          between trip starts on the route between
          ``headway_start_time`` and ``headway_end_time`` on the given
          dates
        - ``'min_headway'``: minimum of the durations (in minutes)
          mentioned above
        - ``'mean_headway'``: mean of the durations (in minutes)
          mentioned above
        - ``'peak_num_trips'``: maximum number of simultaneous trips in
          service (for the given direction, or for both directions when
          ``split_directions==False``)
        - ``'peak_start_time'``: start time of first longest period
          during which the peak number of trips occurs
        - ``'peak_end_time'``: end time of first longest period during
          which the peak number of trips occurs
        - ``'service_duration'``: total of the duration of each trip on
          the route in the given subset of trips; measured in hours
        - ``'service_distance'``: total of the distance traveled by each
          trip on the route in the given subset of trips; measured in
          whatever distance units are present in ``trip_stats_subset``;
          contains all ``np.nan`` entries if ``feed.shapes is None``
        - ``'service_speed'``: service_distance/service_duration;
          measured in distance units per hour
        - ``'mean_trip_distance'``: service_distance/num_trips
        - ``'mean_trip_duration'``: service_duration/num_trips
        - ``'start_stop_id'``: ``start_stop_id`` of the first trip for the route
        - ``'end_stop_id'``: ``end_stop_id`` of the first trip for the route
        - ``'num_stops'``: ``num_stops`` of the first trip for the route
        - ``'start_zone'``: ``start_zone`` of the first trip for the route
        - ``'end_zone'``: ``end_zone`` of the first trip for the route
        
        If not ``split_directions``, then remove the
        direction_id column and compute each route's stats,
        except for headways, using
        its trips running in both directions.
        In this case, (1) compute max headway by taking the max of the
        max headways in both directions; (2) compute mean headway by
        taking the weighted mean of the mean headways in both
        directions.
        
        If ``trip_stats_subset`` is empty, return an empty DataFrame.

    """
    f = trip_stats_subset.copy()
    f[['start_time', 'end_time']] = f[['start_time', 'end_time']
      ].applymap(gtfstk.helpers.timestr_to_seconds)

    headway_start = gtfstk.helpers.timestr_to_seconds(headway_start_time)
    headway_end = gtfstk.helpers.timestr_to_seconds(headway_end_time)
    
    def compute_route_stats(group):
        d = OrderedDict()
        d['route_short_name'] = group['route_short_name'].iat[0]
        d['route_type'] = group['route_type'].iat[0]
        d['num_trips'] = group.shape[0]
        d['num_trip_starts'] = group['start_time'].count()
        d['num_trip_ends'] = group.loc[
          group['end_time'] < 24*3600, 'end_time'].count()
        d['is_loop'] = int(group['is_loop'].any())
        d['is_bidirectional'] = int(group['direction_id'].unique().size > 1)
        d['start_time'] = group['start_time'].min()
        d['end_time'] = group['end_time'].max()

        # Compute headway stats
        headways = np.array([])
        for direction in [0, 1]:
            stimes = group[group['direction_id'] == direction][
              'start_time'].values
            stimes = sorted([stime for stime in stimes
              if headway_start <= stime <= headway_end])
            headways = np.concatenate([headways, np.diff(stimes)])
        if headways.size:
            d['max_headway'] = np.max(headways)/60  # minutes
            d['min_headway'] = np.min(headways)/60  # minutes
            d['mean_headway'] = np.mean(headways)/60  # minutes
        else:
            d['max_headway'] = np.nan
            d['min_headway'] = np.nan
            d['mean_headway'] = np.nan

        # Compute peak num trips
        times = np.unique(group[['start_time', 'end_time']].values)
        counts = [gtfstk.helpers.count_active_trips(group, t) for t in times]
        start, end = gtfstk.helpers.get_peak_indices(times, counts)
        d['peak_num_trips'] = counts[start]
        d['peak_start_time'] = times[start]
        d['peak_end_time'] = times[end]

        d['service_distance'] = group['distance'].sum()
        d['service_duration'] = group['duration'].sum()

        # Added by cjer
        d['start_stop_id'] = group['start_stop_id'].iat[0]
        d['end_stop_id'] = group['end_stop_id'].iat[0]

        d['num_stops'] = group['num_stops'].iat[0]

        d['start_zone'] = group['start_zone'].iat[0]
        d['end_zone'] = group['end_zone'].iat[0]


        return pd.Series(d)
    
    g = f.groupby('route_id').apply(
        compute_route_stats).reset_index()

    # Compute a few more stats
    g['service_speed'] = g['service_distance']/g['service_duration']
    g['mean_trip_distance'] = g['service_distance']/g['num_trips']
    g['mean_trip_duration'] = g['service_duration']/g['num_trips']
    
    # Convert route times to time strings
    g[['start_time', 'end_time', 'peak_start_time', 'peak_end_time']] =\
        g[['start_time', 'end_time', 'peak_start_time', 'peak_end_time']].\
            applymap(lambda x: gtfstk.helpers.timestr_to_seconds(x, inverse=True))
    
    g['service_speed'] = g.service_speed/1000
    g = (g
         .merge(feed.routes[['route_id', 'route_long_name', 'agency_id']], how='left', on='route_id')
         .merge(feed.agency[['agency_id', 'agency_name']], how='left', on='agency_id')
        )
    g = g[['route_id', 'route_short_name', 'agency_id', 'agency_name', 'route_long_name', 'route_type', 
           'num_trips', 'num_trip_starts', 'num_trip_ends', 'is_loop', 
           'is_bidirectional', 'start_time', 'end_time', 'max_headway', 'min_headway', 
           'mean_headway', 'peak_num_trips', 'peak_start_time', 'peak_end_time',
           'service_distance', 'service_duration', 'service_speed',
           'mean_trip_distance', 'mean_trip_duration', 'start_stop_id',
           'end_stop_id', 'num_stops', 'start_zone', 'end_zone', 
           ]]
    
    return g

In [19]:
ts = compute_trip_stats_partridge(feed, zones)

In [20]:
rs = compute_route_stats_base_partridge(ts)

In [27]:
OUTPUT_DIR = 'data/gtfs_stats/'
os.mkdir(OUTPUT_DIR)
date_str = '2018-03-05'
ts.to_pickle(OUTPUT_DIR+date_str+'_trip_stats.pkl.gz', compression='gzip')

In [28]:
pd.read_pickle(OUTPUT_DIR+date_str+'_trip_stats.pkl.gz')

,trip_id,route_id,route_short_name,route_type,direction_id,shape_id,num_stops,start_time,end_time,start_stop_id,...,start_stop_code,end_stop_code,start_stop_name,end_stop_name,start_zone,end_zone,is_loop,duration,distance,speed
0,10096398_040318,9735,70,3,1,88862,37,21:00:00,22:05:22,34657,...,63436,63275,קניון קרני שומרון,אוניברסיטת אריאל/כביש 31,השומרון,השומרון,0,1.089444,45786,42.026925
1,10096399_040318,9735,70,3,1,88862,37,23:00:00,24:05:22,34657,...,63436,63275,קניון קרני שומרון,אוניברסיטת אריאל/כביש 31,השומרון,השומרון,0,1.089444,45786,42.026925
2,10221383_040318,9194,102,3,1,69510,16,14:45:00,15:21:28,34583,...,63362,63433,קופת חולים כללית,קופת חולים לאומית/דרך הנחשונים,השומרון,השומרון,0,0.607778,30736,50.571115
3,10221384_040318,9194,102,3,1,69510,16,15:50:00,16:26:28,34583,...,63362,63433,קופת חולים כללית,קופת חולים לאומית/דרך הנחשונים,השומרון,השומרון,0,0.607778,30736,50.571115
4,10230038_040318,11169,101,3,1,84074,31,15:30:00,16:26:40,29309,...,20001,34138,מסוף 2000,בריכה/עמק דותן,גוש דן,חבל מודיעין,0,0.944444,44754,47.386588
5,10253006_040318,11286,115,3,1,91096,46,05:05:00,05:36:13,17318,...,47222,42657,בית התעשייה,תחנה מרכזית חוף כרמל/הורדה,סובב חיפה,סובב חיפה,0,0.520278,12080,23.218366
6,10292410_040318,9631,291,3,0,84207,13,08:10:00,08:43:40,6382,...,59054,54154,מגאר ג'/מרכז,סחנין משרד הרישוי,כרמיאל,כרמיאל,0,0.561111,14414,25.688317
7,10292416_040318,9631,291,3,0,84207,13,15:30:00,16:03:40,6382,...,59054,54154,מגאר ג'/מרכז,סחנין משרד הרישוי,כרמיאל,כרמיאל,0,0.561111,14414,25.688317
8,10320309_050318,10490,51,3,0,93013,37,09:30:00,09:58:03,10414,...,3242,5677,ברנדייס,מעלה הזיתים,סובב ירושלים,סובב ירושלים,0,0.467500,10027,21.448128
9,10320310_050318,10490,51,3,0,93013,37,13:40:00,14:08:03,10414,...,3242,5677,ברנדייס,מעלה הזיתים,סובב ירושלים,סובב ירושלים,0,0.467500,10027,21.448128


In [29]:
rs.to_pickle(OUTPUT_DIR+date_str+'_route_stats.pkl.gz', compression='gzip')

In [ ]:
import datetime
import os
import re

def batch_stats(folder, output_folder):
    for file in os.listdir(folder):
        date_str = file.split('.')[0]
        date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()
        feed = get_partridge_feed_by_date(folder+file, date)
        zones = get_zones_df(LOCAL_TARIFF_PATH)
        ts = compute_trip_stats_partridge(feed, zones)
        ts.to_pickle(output_folder+date_str+'_trip_stats.pkl.gz', compression='gzip')
        rs = compute_route_stats_base_partridge(ts)
        rs.to_pickle(output_folder+date_str+'_route_stats.pkl.gz', compression='gzip')

GTFS_FEEDS_PATH = 'data/gtfs_feeds/'
OUTPUT_DIR = 'data/gtfs_stats/'

batch_stats(GTFS_FEEDS_PATH, OUTPUT_DIR)


In [ ]:
FILE_NAME_RE = re.compile('^(?P<date_str>[^_]+?)_(?P<type>\w+)\.pkl.gz')

STATS_TYPES = ['trip_stats', 'route_stats']

existing_files = [(g[0], g[1]) for g in 
                  (re.match(FILE_NAME_RE, file).groups() for file in os.listdir(OUTPUT_DIR))]

## What's next

TODO

1. put this all back into proper documented functions
1. add date
1. integrate with custom day cutoff
1. add logging
1. write tests
1. add split_directions
1. add time between stops - max, min, mean (using delta)
1. add day and night headways and num_trips (maybe noon also)
1. create a function for converting this to a timeseries good for pandas


